## Config .env
DB_USER=user_securite_routiere
DB_PASSWORD=password123
DB_HOST=localhost
DB_PORT=5432
DB_NAME=securite_routiere_db

Une fois que le fichier .env est bien configuré on peut lancer ça et commencer à travailler

In [1]:
import sys
sys.path.append("..")  # remonte d'un niveau vers le dossier parent
from utils.utils import connect_to_db, insert_data_to_db, mapping_colonnes, fetch_data_from_db
engine = connect_to_db()
df = fetch_data_from_db(table_name='accidents_cleaned', engine=engine, schema='silver')
df

✅ Connexion à la base PostgreSQL réussie !
OK: "silver"."accidents_cleaned"


,num_acc,datetime,nom_com,an,mois,jour,hrmn,lum,agg,int,...,year_georef,com_name,dep_code,dep_name,epci_code,epci_name,reg_code,reg_name,com_arm_name,code_com
0,201600055393,2016-03-14 08:20:00+00:00,Paris-17e-arrondissement,2016,3,14,09:20,Plein jour,En agglomération,2,...,2016,Paris,75,Paris,200054781,Métropole du Grand Paris,11,Île-de-France,Paris 17e Arrondissement,75117
1,201600056285,2016-08-03 05:40:00+00:00,Paris-19e-arrondissement,2016,8,3,07:40,Plein jour,En agglomération,2,...,2016,Paris,75,Paris,200054781,Métropole du Grand Paris,11,Île-de-France,Paris 19e Arrondissement,75119
2,201600056621,2016-09-22 10:25:00+00:00,Paris-20e-arrondissement,2016,9,22,12:25,Plein jour,En agglomération,1,...,2016,Paris,75,Paris,200054781,Métropole du Grand Paris,11,Île-de-France,Paris 20e Arrondissement,75120
3,201600057859,2016-09-14 15:15:00+00:00,La Plaine-des-palmistes,2016,9,14,17:15,Plein jour,Hors agglomération,1,...,2016,La Plaine-des-Palmistes,974,La Réunion,249740093,CA Intercommunale de la Réunion Est (Cirest),04,La Réunion,La Plaine-des-Palmistes,97406
4,201400013574,2014-01-27 17:45:00+00:00,Aubagne,2014,1,27,18:45,Nuit avec éclairage public allumé,En agglomération,3,...,2015,Aubagne,13,Bouches-du-Rhône,241300268,CA du Pays d'Aubagne et de l'Etoile (Capae),93,Provence-Alpes-Côte d'Azur,Aubagne,13005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475906,201600049674,2016-01-28 03:40:00+00:00,Canejan,2016,1,28,04:40,Nuit sans éclairage public,Hors agglomération,1,...,2016,Canéjan,33,Gironde,243301165,CC Jalle-Eau-Bourde,75,Aquitaine-Limousin-Poitou-Charentes,Canéjan,33090
475907,201600049793,2016-07-13 08:25:00+00:00,Gradignan,2016,7,13,10:25,Plein jour,Hors agglomération,1,...,2016,Gradignan,33,Gironde,243300316,Bordeaux Métropole,75,Aquitaine-Limousin-Poitou-Charentes,Gradignan,33192
475908,201600050415,2016-09-08 05:45:00+00:00,Toulon,2016,9,8,07:45,Plein jour,Hors agglomération,1,...,2016,Toulon,83,Var,248300543,CA Toulon Provence Méditerranée,93,Provence-Alpes-Côte d'Azur,Toulon,83137
475909,201600027375,2016-02-06 23:05:00+00:00,Villeneuve-sur-lot,2016,2,7,00:05,Nuit sans éclairage public,Hors agglomération,3,...,2016,Villeneuve-sur-Lot,47,Lot-et-Garonne,200023307,CA du Grand Villeneuvois,75,Aquitaine-Limousin-Poitou-Charentes,Villeneuve-sur-Lot,47323


In [9]:
df['lat'].head()

0    4887440.0
1    4887840.0
2    4886710.0
3          NaN
4    4327987.0
Name: lat, dtype: float64

In [10]:
import pandas as pd

df_silver = df.copy()  # Supposons que df_silver est le DataFrame nettoyé après vos étapes de nettoyage.

# =====================================================================
# BLOC DE DIAGNOSTIC COMPLET SUR LE DATAFRAME SILVER (df_silver)
# Objectif : Valider le nettoyage et préparer la couche Gold.
# =====================================================================

# Prérequis : Le DataFrame 'df_silver' doit exister.
if 'df_silver' in locals() and df_silver is not None:
    print("=======================================================================")
    print("        Diagnostic Complet des Données Nettoyées (df_silver)         ")
    print("=======================================================================")

    # --- 1. Analyse de la complétion (Taux de remplissage) ---
    print("\n--- 1. Taux de Remplissage par Colonne (de la pire à la meilleure) ---")
    
    total_lignes = len(df_silver)
    comptage_non_na = df_silver.count()
    taux_remplissage = (comptage_non_na / total_lignes) * 100
    
    stats_remplissage = pd.DataFrame({'Taux_Remplissage': taux_remplissage})
    stats_remplissage['Taux_Remplissage'] = stats_remplissage['Taux_Remplissage'].map('{:.2f} %'.format)
    print(stats_remplissage.sort_values(by='Taux_Remplissage').to_string())
    
    seuil_suppression = 10.0
    colonnes_a_supprimer = taux_remplissage[taux_remplissage < seuil_suppression].index.tolist()
    if colonnes_a_supprimer:
        print(f"\n💡 Suggestion : Les colonnes suivantes sont remplies à moins de {seuil_suppression}% et pourraient être supprimées avant la couche Gold :")
        print(colonnes_a_supprimer)


    # --- 2. Analyse de la Cardinalité (Nombre de valeurs uniques) ---
    print("\n\n--- 2. Cardinalité (Nombre de Valeurs Uniques) par Colonne ---")
    
    cardinalite = df_silver.nunique()
    stats_cardinalite = pd.DataFrame({'Nombre_Valeurs_Uniques': cardinalite})
    print(stats_cardinalite.sort_values(by='Nombre_Valeurs_Uniques', ascending=False).to_string())

    colonnes_inutiles = cardinalite[cardinalite == 1].index.tolist()
    if colonnes_inutiles:
        print("\n💡 Suggestion : Les colonnes suivantes n'ont qu'une seule valeur et sont probablement inutiles :")
        print(colonnes_inutiles)


    # --- 3. Description Statistique des Colonnes Numériques ---
    print("\n\n--- 3. Description Statistique des Colonnes Numériques ---")
    
    # On sélectionne les colonnes numériques après votre nettoyage
    df_numerique = df_silver.select_dtypes(include=['int64', 'float64', 'Int64', 'Float64'])
    if not df_numerique.empty:
        # .describe() calcule les stats, .round(2) arrondit à 2 décimales.
        print(df_numerique.describe().round(2).to_string())
    else:
        print("Aucune colonne numérique trouvée pour l'analyse.")
            
    print("\n=======================================================================")

else:
    print("❌ Erreur : Le DataFrame 'df_silver' n'a pas été trouvé. Veuillez d'abord l'importer et le nettoyer.")

        Diagnostic Complet des Données Nettoyées (df_silver)         

--- 1. Taux de Remplissage par Colonne (de la pire à la meilleure) ---
             Taux_Remplissage
occutc                 1.26 %
v1                    10.56 %
num_veh              100.00 %
grav                 100.00 %
sexe                 100.00 %
catr                 100.00 %
catv                 100.00 %
year_georef          100.00 %
com                  100.00 %
dep                  100.00 %
col                  100.00 %
num_acc              100.00 %
int                  100.00 %
datetime             100.00 %
an                   100.00 %
mois                 100.00 %
jour                 100.00 %
hrmn                 100.00 %
catu                 100.00 %
agg                  100.00 %
lum                  100.00 %
infra                 11.45 %
an_nais               18.95 %
obs                   21.30 %
locp                  21.40 %
etatp                 27.89 %
v2                     4.10 %
date              

In [24]:
df['coordonnees'].head()

0      48.887702, 2.304844
1       48.87834, 2.374525
2      48.867131, 2.397971
3    -21.152906, 55.642371
4       43.27476, 5.565483
Name: coordonnees, dtype: object

In [11]:
import pandas as pd 

# =====================================================================
# DIAGNOSTIC ET AMÉLIORATION DES COORDONNÉES GPS DANS df_silver
# =====================================================================

# Prérequis : Le DataFrame 'df_silver' doit exister en mémoire.
if 'df_silver' not in locals() or df_silver is None:
    print("❌ Erreur : Le DataFrame 'df_silver' n'a pas été trouvé en mémoire.")
else:
    print("DataFrame 'df_silver' trouvé. Début du processus d'amélioration des coordonnées GPS...")

    # --- ÉTAPE 1 : Diagnostic Avant Amélioration ---
    # ... (cette partie ne change pas)
    total_lignes = len(df_silver)
    nan_avant_lat = df_silver['lat'].isna().sum()
    pourcentage_avant_lat = (nan_avant_lat / total_lignes) * 100
    print(f"\nAvant amélioration (lat) : {nan_avant_lat:,} valeurs manquantes ({pourcentage_avant_lat:.2f} %).")
    nan_avant_long = df_silver['long'].isna().sum()
    pourcentage_avant_long = (nan_avant_long / total_lignes) * 100
    print(f"Avant amélioration (long): {nan_avant_long:,} valeurs manquantes ({pourcentage_avant_long:.2f} %).")

    # --- ÉTAPE 2 : Remplissage des Coordonnées Manquantes (Logique Corrigée) ---
    # LA CORRECTION EST ICI : On vérifie si 'lat' OU '|' 'long' est vide.
    # Les parenthèses sont importantes pour l'ordre des opérations.
    condition_remplissage = (df_silver['lat'].isna() | df_silver['long'].isna()) & df_silver['coordonnees'].notna()
    
    nb_lignes_a_remplir = condition_remplissage.sum()
    print(f"\nTrouvé {nb_lignes_a_remplir:,} lignes où au moins une coordonnée (lat/long) peut être récupérée.")

    if nb_lignes_a_remplir > 0:
        coords_split = df_silver.loc[condition_remplissage, 'coordonnees'].str.split(',', expand=True)
        
        if coords_split.shape[1] >= 2:
            # On remplit 'lat' UNIQUEMENT si elle est vide
            df_silver.loc[condition_remplissage & df_silver['lat'].isna(), 'lat'] = coords_split[0]
            # On remplit 'long' UNIQUEMENT si elle est vide
            df_silver.loc[condition_remplissage & df_silver['long'].isna(), 'long'] = coords_split[1]
    
            # --- ÉTAPE 3 : Reconversion des Types ---
            df_silver['lat'] = pd.to_numeric(df_silver['lat'], errors='coerce')
            df_silver['long'] = pd.to_numeric(df_silver['long'], errors='coerce')

            # --- ÉTAPE 4 : Diagnostic Après Amélioration ---
            # ... (cette partie ne change pas)
            nan_apres_lat = df_silver['lat'].isna().sum()
            pourcentage_apres_lat = (nan_apres_lat / total_lignes) * 100
            print(f"\nAprès amélioration (lat) : {nan_apres_lat:,} valeurs manquantes ({pourcentage_apres_lat:.2f} %).")
            nan_apres_long = df_silver['long'].isna().sum()
            pourcentage_apres_long = (nan_apres_long / total_lignes) * 100
            print(f"Après amélioration (long): {nan_apres_long:,} valeurs manquantes ({pourcentage_apres_long:.2f} %).")
            
            print("\n✅ Succès ! Le DataFrame 'df_silver' en mémoire a été mis à jour.")
        else:
            print("⚠️ Attention : La découpe de la colonne 'coordonnees' n'a pas produit 2 colonnes. Le remplissage est annulé.")
    else:
        print("\n-> Aucune coordonnée supplémentaire à remplir.")

DataFrame 'df_silver' trouvé. Début du processus d'amélioration des coordonnées GPS...

Avant amélioration (lat) : 233,966 valeurs manquantes (49.16 %).
Avant amélioration (long): 233,966 valeurs manquantes (49.16 %).

Trouvé 218,808 lignes où au moins une coordonnée (lat/long) peut être récupérée.


/var/folders/4_/cb0dgr6d07d_36m06pf2rqjr0000gn/T/ipykernel_98449/2035906928.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['-21.152906' '43.331442' '16.257101' ... '45.740286' '43.124011'
 '48.888312']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_silver.loc[condition_remplissage & df_silver['lat'].isna(), 'lat'] = coords_split[0]
/var/folders/4_/cb0dgr6d07d_36m06pf2rqjr0000gn/T/ipykernel_98449/2035906928.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[' 55.642371' ' 5.398574' ' -61.598425' ... ' 4.888678' ' 5.930466'
 ' 2.164508']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_silver.loc[condition_remplissage & df_silver['long'].isna(), 'long'] = coords_split[1]



Après amélioration (lat) : 15,206 valeurs manquantes (3.20 %).
Après amélioration (long): 15,202 valeurs manquantes (3.19 %).

✅ Succès ! Le DataFrame 'df_silver' en mémoire a été mis à jour.


In [12]:
df_silver[['lat','long']].head()

,lat,long
0,4.887440e+06,229293.000000
1,4.887840e+06,237423.000000
2,4.886710e+06,239801.000000
3,-2.115291e+01,55.642371
4,4.327987e+06,556520.000000


In [ ]:
df_silver = df.copy()
# =====================================================================
# BLOC COMPLET DE NETTOYAGE DES COORDONNÉES DANS df_silver
# =====================================================================
import pandas as pd
import geopandas
import numpy as np

# Prérequis : Le DataFrame 'df_silver' doit exister (après renommage).
if 'df_silver' not in locals() or df_silver is None:
    print("❌ Erreur : Le DataFrame 'df_silver' n'a pas été trouvé en mémoire.")
else:
    print("DataFrame 'df_silver' trouvé. Début du processus complet de nettoyage des coordonnées.")

    # --- ÉTAPE 1 : VOTRE SCRIPT DE REMPLISSAGE DES NAN ---
    print("\n--- Étape 1: Remplissage des NaN à partir de 'coordonnees' ---")
    condition_remplissage = (df_silver['lat'].isna() | df_silver['long'].isna()) & df_silver['coordonnees'].notna()
    if condition_remplissage.sum() > 0:
        coords_split = df_silver.loc[condition_remplissage, 'coordonnees'].str.split(',', expand=True)
        if coords_split.shape[1] >= 2:
            df_silver.loc[condition_remplissage, 'lat'] = coords_split[0]
            df_silver.loc[condition_remplissage, 'long'] = coords_split[1]
    
    # On s'assure que tout est numérique pour la suite
    df_silver['lat'] = pd.to_numeric(df_silver['lat'], errors='coerce')
    df_silver['long'] = pd.to_numeric(df_silver['long'], errors='coerce')
    print("-> Remplissage des NaN terminé. Les colonnes contiennent maintenant un mélange de formats.")

    # --- ÉTAPE 2 : STANDARDISATION DES COORDONNÉES AU FORMAT WGS 84 ---
    print("\n--- Étape 2: Standardisation de toutes les coordonnées au format WGS 84 (GPS) ---")
    # On identifie les lignes qui sont au format Lambert (valeur absolue de latitude > 90)
    condition_lambert = df_silver['lat'].notna() & (df_silver['lat'].abs() > 90)
    
    print(f"-> Trouvé {condition_lambert.sum():,} lignes au format Lambert à convertir.")

    if condition_lambert.sum() > 0:
        # On crée un GeoDataFrame uniquement avec les lignes à convertir
        gdf_lambert = geopandas.GeoDataFrame(
            df_silver[condition_lambert], 
            geometry=geopandas.points_from_xy(df_silver[condition_lambert].long, df_silver[condition_lambert].lat),
            crs="EPSG:2154"  # On déclare que le système d'origine est Lambert-93
        )
        # On effectue la conversion vers WGS 84
        gdf_wgs_84_converted = gdf_lambert.to_crs("EPSG:4326")
        
        # ON ÉCRASE les anciennes valeurs Lambert par les nouvelles valeurs WGS 84
        df_silver.loc[condition_lambert, 'lat'] = gdf_wgs_84_converted.geometry.y
        df_silver.loc[condition_lambert, 'long'] = gdf_wgs_84_converted.geometry.x
        print("-> Conversion terminée.")

    # --- ÉTAPE 3 : VALIDATION FINALE ---
    print("\n--- Étape 3: Validation des résultats ---")
    print("Statistiques des colonnes 'lat' et 'long' après standardisation :")
    print(df_silver[['lat', 'long']].describe().round(6))
    
    print("\n✅ Succès ! 'df_silver' contient maintenant des coordonnées 'lat' et 'long' standardisées.")

In [3]:
df_silver[['lat','long']].head()

,lat,long
0,31.106874,-1.778833
1,31.114616,-1.696731
2,31.105999,-1.671965
3,-21.152906,55.642371
4,26.386748,1.639658


In [37]:
col_commune = [
        'com',
        'code_com',
        'nom_com',
        'com_name',
        'com_arm_name',
        'adr',       
        'lat',
        'long',       
        'code_postal',
        'coordonnees',
        'num'
    ]

    # --- ÉTAPE 2 : Créer le nouveau DataFrame ---
df_commune = df[col_commune].copy()

In [38]:
df_commune.head()

,com,code_com,nom_com,com_name,com_arm_name,adr,lat,long,code_postal,coordonnees,num
0,117,75117,Paris-17e-arrondissement,Paris,Paris 17e Arrondissement,"2, AVENUE DE LA GRANDE A",4887440.0,229293.0,75017,"48.887702, 2.304844",2
1,119,75119,Paris-19e-arrondissement,Paris,Paris 19e Arrondissement,"30-32, AVENUE MATHURIN M",4887840.0,237423.0,75019,"48.87834, 2.374525",30
2,120,75120,Paris-20e-arrondissement,Paris,Paris 20e Arrondissement,"55, RUE ORFILA",4886710.0,239801.0,75020,"48.867131, 2.397971",55
3,406,97406,La Plaine-des-palmistes,La Plaine-des-Palmistes,La Plaine-des-Palmistes,None,NaN,NaN,97431,"-21.152906, 55.642371",None
4,005,13005,Aubagne,Aubagne,Aubagne,FENESTRELLES (CHEMIN DE),4327987.0,556520.0,13400,"43.27476, 5.565483",None


In [44]:
# =======================================================
# CRÉATION DU DATAFRAME df_commune
# =======================================================

# Prérequis : Le DataFrame 'df_silver' doit exister.
if 'df_silver' not in locals() or df_silver is None:
    print("❌ Erreur : Le DataFrame 'df_silver' n'a pas été trouvé.")
else:
    print("DataFrame 'df_silver' trouvé. Création du DataFrame 'df_commune'...")

    # --- ÉTAPE 1 : Définir la liste des colonnes pour la commune ---
    # On prend toutes les colonnes qui décrivent une commune de manière unique.
    col_commune = [
        'com',
        'code_com',
        'nom_com',
        'com_name',
        'com_arm_name',
        'adr',       
        'lat',
        'long',       
        'code_postal',
        'coordonnees',
        'num'
    ]

    # --- ÉTAPE 2 : Créer le nouveau DataFrame ---
    df_commune = df_silver[col_commune].copy()

    # --- ÉTAPE 3 : Dédoublonner et nettoyer ---
    # La clé primaire de la table commune est le code commune.
    # On ne garde donc qu'une seule ligne par code commune.
    df_commune.dropna(subset=['com'], inplace=True)
    df_commune.drop_duplicates(subset=['com'], inplace=True)
    
    # --- ÉTAPE 4 : Renommer la colonne clé pour correspondre au SQL ---
    df_commune.rename(columns={'code_com': 'com_code'}, inplace=True)

    # --- ÉTAPE 5 : Afficher le résultat ---
    print("\n✅ DataFrame 'df_commune' créé avec succès !")
    print(f"Nombre de lignes (communes uniques) : {len(df_commune):,}")
    print("\nAperçu des 5 premières lignes de df_commune :")
   

DataFrame 'df_silver' trouvé. Création du DataFrame 'df_commune'...

✅ DataFrame 'df_commune' créé avec succès !
Nombre de lignes (communes uniques) : 12,335

Aperçu des 5 premières lignes de df_commune :


In [46]:
df_commune.head()

,com,com_code,nom_com,com_name,com_arm_name,adr,lat,long,code_postal,coordonnees,num
0,117,75117,Paris-17e-arrondissement,Paris,Paris 17e Arrondissement,"2, AVENUE DE LA GRANDE A",4.887440e+06,229293.000000,75017,"48.887702, 2.304844",2
1,119,75119,Paris-19e-arrondissement,Paris,Paris 19e Arrondissement,"30-32, AVENUE MATHURIN M",4.887840e+06,237423.000000,75019,"48.87834, 2.374525",30
2,120,75120,Paris-20e-arrondissement,Paris,Paris 20e Arrondissement,"55, RUE ORFILA",4.886710e+06,239801.000000,75020,"48.867131, 2.397971",55
3,406,97406,La Plaine-des-palmistes,La Plaine-des-Palmistes,La Plaine-des-Palmistes,None,-2.115291e+01,55.642371,97431,"-21.152906, 55.642371",None
4,005,13005,Aubagne,Aubagne,Aubagne,FENESTRELLES (CHEMIN DE),4.327987e+06,556520.000000,13400,"43.27476, 5.565483",None


In [47]:
# =======================================================
# CRÉATION DU DATAFRAME df_vehicule
# =======================================================

# Prérequis : Le DataFrame 'df_silver' doit exister.    
if 'df_silver' not in locals() or df_silver is None:
    print("❌ Erreur : Le DataFrame 'df' n'a pas été trouvé.")
else:
    print("\nDataFrame 'df_silver' trouvé. Création du DataFrame 'df_vehicule'...")    

    # --- ÉTAPE 1 : Définir la liste des colonnes pour le véhicule ---
    col_vehicule = [
        'num_veh',
        'place',
        'choc',
        'manv',
        'catv',
        'secu'
    ]

    # --- ÉTAPE 2 : Créer le nouveau DataFrame ---
    df_vehicule = df_silver[col_vehicule].copy()

    


DataFrame 'df_silver' trouvé. Création du DataFrame 'df_vehicule'...


In [12]:
df_vehicule.head()  

,num_veh,place,choc,manv,catv,secu
0,"A01,B01","1,1","Avant,Arrière","Sans changement de direction,Arrêté (hors stat...","VL seul,Scooter > 50 cm3 et <= 125 cm3","Casque,Ceinture"
1,A01,1,Avant gauche,Tournant A gauche,VL seul,Ceinture
2,"B01,A01","1,1","Avant,Avant gauche","Sans changement de direction,En s’insérant","Motocyclette > 125 cm3,VL seul","Casque,Ceinture"
3,"B02,A01","3,3,1,2,1","Avant,Avant","Sans changement de direction,Sans changement d...","VL seul,VU seul 1","Dispositif enfants,Ceinture,Ceinture,Ceinture,..."
4,"A01,B01","1,1,2","Avant,Avant droit","Sans changement de direction,Sans changement d...","Scooter < 50 cm3,VL seul","Ceinture,Casque,Casque"


In [48]:
# =====================================================================
# BLOC FINAL POUR LA CRÉATION DE df_vehicule AVEC CLÉ COMPOSITE Id_veh
# =====================================================================

# Prérequis : Le DataFrame 'df_silver' doit exister.
if 'df_silver' not in locals() or df_silver is None:
    print("❌ Erreur : Le DataFrame 'df_silver' n'a pas été trouvé.")
else:
    print("DataFrame 'df_silver' trouvé. Préparation de 'df_vehicule'...")

    # --- ÉTAPE 1 : Nettoyage Profond de df ---
    # (Cette partie ne change pas)
    colonnes_a_nettoyer = ['num_veh', 'place', 'choc', 'manv', 'catv', 'secu']
    print("\nNettoyage profond des colonnes contenant des listes...")
    for colonne in colonnes_a_nettoyer:
        df_silver[colonne] = df_silver[colonne].astype(str).str.split(',').str[0].str.strip()
        print(f"-> Colonne '{colonne}' nettoyée.")

    # --- ÉTAPE 2 : Sélection des Colonnes ---
    # On sélectionne les colonnes de base nécessaires pour créer la clé et décrire le véhicule.
    col_vehicule = [
        'num_acc',
        'num_veh',
        'place',
        'choc',
        'manv',
        'catv',
        'secu'
    ]
    print("\nCréation du DataFrame 'df_vehicule' à partir des données nettoyées...")
    df_vehicule = df_silver[col_vehicule].copy()
    
    # --- ÉTAPE 3 : Création de la Clé Primaire Composite 'Id_veh' ---
    print("\nCréation de la clé primaire composite 'id_veh'...")
    
    # On s'assure que les deux colonnes sont bien des chaînes de caractères avant de les concaténer.
    df_vehicule['num_acc'] = df_vehicule['num_acc'].astype(str)
    df_vehicule['num_veh'] = df_vehicule['num_veh'].astype(str)
    
    # On combine 'num_acc' et 'num_veh' pour créer la clé unique.
    # Exemple : num_acc='201900000001' et num_veh='A01' -> Id_veh='201900000001_A01'
    df_vehicule['id_veh'] = df_vehicule['num_acc'] + '_' + df_vehicule['num_veh']
    print("-> Clé 'id_veh' créée avec succès.")

    # --- ÉTAPE 4 : Application des Contraintes d'Unicité ---
    print("\nApplication des contraintes (nettoyage et dédoublonnage)...")
    
    # On supprime les lignes où les composants de la clé sont invalides.
    # C'est important de le faire avant de dédoublonner.
    df_vehicule.dropna(subset=['num_acc', 'num_veh'], inplace=True)
    
    # On dédoublonne maintenant sur notre nouvelle clé unique 'Id_veh'.
    # Cela a le même effet que de dédoublonner sur la paire, mais c'est plus explicite.
    df_vehicule.drop_duplicates(subset=['id_veh'], inplace=True)
    
    # --- ÉTAPE 5 : Finalisation du DataFrame ---
    # On peut réorganiser les colonnes pour mettre la clé en premier, c'est plus propre.
    colonnes_finales = [
        'id_veh',         # On rajoute 'num_acc' pour la clé étrangère
        'num_acc',        # On garde les composants originaux pour la traçabilité
        'num_veh', 
        'place', 
        'choc', 
        'manv', 
        'catv', 
        'secu'
    ]
    df_vehicule = df_vehicule[colonnes_finales]
    
    # --- ÉTAPE 6 : Validation Finale ---
    print("\n✅ DataFrame 'df_vehicule' final créé avec succès !")
    print(f"Nombre de lignes finales (véhicules uniques) : {len(df_vehicule):,}")
    
    print("\nAperçu du DataFrame 'df_vehicule' final avec la nouvelle clé 'id_veh':")
    print(df_vehicule.head())

DataFrame 'df_silver' trouvé. Préparation de 'df_vehicule'...

Nettoyage profond des colonnes contenant des listes...
-> Colonne 'num_veh' nettoyée.
-> Colonne 'place' nettoyée.
-> Colonne 'choc' nettoyée.
-> Colonne 'manv' nettoyée.
-> Colonne 'catv' nettoyée.
-> Colonne 'secu' nettoyée.

Création du DataFrame 'df_vehicule' à partir des données nettoyées...

Création de la clé primaire composite 'id_veh'...
-> Clé 'id_veh' créée avec succès.

Application des contraintes (nettoyage et dédoublonnage)...

✅ DataFrame 'df_vehicule' final créé avec succès !
Nombre de lignes finales (véhicules uniques) : 475,911

Aperçu du DataFrame 'df_vehicule' final avec la nouvelle clé 'id_veh':
             id_veh       num_acc num_veh place          choc  \
0  201600055393_A01  201600055393     A01     1         Avant   
1  201600056285_A01  201600056285     A01     1  Avant gauche   
2  201600056621_B01  201600056621     B01     1         Avant   
3  201600057859_B02  201600057859     B02     3      

In [49]:
df_vehicule.head()

,id_veh,num_acc,num_veh,place,choc,manv,catv,secu
0,201600055393_A01,201600055393,A01,1,Avant,Sans changement de direction,VL seul,Casque
1,201600056285_A01,201600056285,A01,1,Avant gauche,Tournant A gauche,VL seul,Ceinture
2,201600056621_B01,201600056621,B01,1,Avant,Sans changement de direction,Motocyclette > 125 cm3,Casque
3,201600057859_B02,201600057859,B02,3,Avant,Sans changement de direction,VL seul,Dispositif enfants
4,201400013574_A01,201400013574,A01,1,Avant,Sans changement de direction,Scooter < 50 cm3,Ceinture


In [50]:
df_vehicule.isnull().sum()

id_veh     0
num_acc    0
num_veh    0
place      0
choc       0
manv       0
catv       0
secu       0
dtype: int64

In [ ]:
df_vehicule.isna().sum()

In [18]:
# =====================================================================
# DÉFINITION DES FONCTIONS DE CRÉATION DES DATAFRAMES
# =====================================================================
import pandas as pd

def creer_df_vehicule(df_source: pd.DataFrame) -> pd.DataFrame:
    """
    Crée le DataFrame des véhicules à partir du DataFrame source (df_silver).
    Nettoie les colonnes, crée une clé primaire 'id_veh', et dédoublonne.
    """
    print("=====================================================")
    print("        Fonction : Création de df_vehicule           ")
    print("=====================================================")
    
    # --- ÉTAPE 1 : Nettoyage Profond ---
    colonnes_a_nettoyer = ['num_veh', 'place', 'choc', 'manv', 'catv', 'secu']
    print("\nNettoyage profond des colonnes...")
    for colonne in colonnes_a_nettoyer:
        df_source[colonne] = df_source[colonne].astype(str).str.split(',').str[0].str.strip()
    print("-> Colonnes nettoyées.")

    # --- ÉTAPE 2 : Sélection et Création du DataFrame ---
    col_vehicule = ['num_acc', 'num_veh', 'place', 'choc', 'manv', 'catv', 'secu']
    df_vehicule = df_source[col_vehicule].copy()
    
    # --- ÉTAPE 3 : Création de la Clé Primaire 'id_veh' ---
    print("\nCréation de la clé primaire 'id_veh'...")
    df_vehicule['num_acc'] = df_vehicule['num_acc'].astype(str)
    df_vehicule['num_veh'] = df_vehicule['num_veh'].astype(str)
    df_vehicule['id_veh'] = df_vehicule['num_acc'] + '_' + df_vehicule['num_veh']
    print("-> Clé 'id_veh' créée.")

    # --- ÉTAPE 4 : Application des Contraintes d'Unicité ---
    df_vehicule.dropna(subset=['num_acc', 'num_veh'], inplace=True)
    df_vehicule.drop_duplicates(subset=['id_veh'], inplace=True)
    
    # --- ÉTAPE 5 : Finalisation du DataFrame ---
    colonnes_finales = ['id_veh', 'num_acc', 'num_veh', 'place', 'choc', 'manv', 'catv', 'secu']
    df_vehicule = df_vehicule[colonnes_finales]
    
    print(f"\n✅ DataFrame 'df_vehicule' créé avec succès ({len(df_vehicule):,} lignes).")
    return df_vehicule


def creer_df_commune(df_source: pd.DataFrame) -> pd.DataFrame:
    """
    Crée le DataFrame des communes à partir du DataFrame source (df_silver).
    Sélectionne les colonnes pertinentes et dédoublonne par code commune.
    """
    print("\n\n=====================================================")
    print("         Fonction : Création de df_commune           ")
    print("=====================================================")
    
    # --- ÉTAPE 1 : Sélection des Colonnes ---

    col_commune = [
        'code_com', 'com_name', 'com_arm_name', 'epci_code', 'dep_code','nom_com', 'adr',
        'lat', 'long', 'code_postal', 'coordonnees', 'com', 'num'
    ]

    df_commune = df_source[col_commune].copy()

    # --- ÉTAPE 2 : Dédoublonnage et Nettoyage ---
    # On dédoublonne sur la colonne 'com' qui semble être la clé la plus fiable ici
    df_commune.dropna(subset=['com'], inplace=True)
    df_commune.drop_duplicates(subset=['com'], inplace=True)
    
    # --- ÉTAPE 3 : Renommage ---
    df_commune.rename(columns={'code_com': 'com_code'}, inplace=True)

    print(f"\n✅ DataFrame 'df_commune' créé avec succès ({len(df_commune):,} lignes).")
    return df_commune


# =====================================================================
# ORCHESTRATION : APPEL DES FONCTIONS
# =====================================================================

# Prérequis : Le DataFrame 'df_silver' doit exister.
if 'df_silver' in locals() and df_silver is not None:
    # On appelle la première fonction pour créer df_vehicule
    df_vehicule = creer_df_vehicule(df_silver)
    
    # On appelle la seconde fonction pour créer df_commune
    df_commune = creer_df_commune(df_silver)
    
    # --- Validation Finale ---
    print("\n\n--- VALIDATION FINALE ---")
    print("Aperçu de df_vehicule :")
    print(df_vehicule.head())
    
    print("\nAperçu de df_commune :")
    print(df_commune.head())
    
else:
    print("❌ Erreur : Le DataFrame 'df_silver' n'a pas été trouvé. Veuillez exécuter les cellules précédentes.")

        Fonction : Création de df_vehicule           

Nettoyage profond des colonnes...
-> Colonnes nettoyées.

Création de la clé primaire 'id_veh'...
-> Clé 'id_veh' créée.

✅ DataFrame 'df_vehicule' créé avec succès (475,911 lignes).


         Fonction : Création de df_commune           

✅ DataFrame 'df_commune' créé avec succès (12,335 lignes).


--- VALIDATION FINALE ---
Aperçu de df_vehicule :
             id_veh       num_acc num_veh place          choc  \
0  201600055393_A01  201600055393     A01     1         Avant   
1  201600056285_A01  201600056285     A01     1  Avant gauche   
2  201600056621_B01  201600056621     B01     1         Avant   
3  201600057859_B02  201600057859     B02     3         Avant   
4  201400013574_A01  201400013574     A01     1         Avant   

                           manv                    catv                secu  
0  Sans changement de direction                 VL seul              Casque  
1             Tournant A gauche                

In [19]:
df_vehicule.head()

,id_veh,num_acc,num_veh,place,choc,manv,catv,secu
0,201600055393_A01,201600055393,A01,1,Avant,Sans changement de direction,VL seul,Casque
1,201600056285_A01,201600056285,A01,1,Avant gauche,Tournant A gauche,VL seul,Ceinture
2,201600056621_B01,201600056621,B01,1,Avant,Sans changement de direction,Motocyclette > 125 cm3,Casque
3,201600057859_B02,201600057859,B02,3,Avant,Sans changement de direction,VL seul,Dispositif enfants
4,201400013574_A01,201400013574,A01,1,Avant,Sans changement de direction,Scooter < 50 cm3,Ceinture


In [20]:
df_commune.head()

,com_code,com_name,com_arm_name,epci_code,dep_code,nom_com,adr,lat,long,code_postal,coordonnees,com,num
0,75117,Paris,Paris 17e Arrondissement,200054781,75,Paris-17e-arrondissement,"2, AVENUE DE LA GRANDE A",4.887440e+06,229293.000000,75017,"48.887702, 2.304844",117,2
1,75119,Paris,Paris 19e Arrondissement,200054781,75,Paris-19e-arrondissement,"30-32, AVENUE MATHURIN M",4.887840e+06,237423.000000,75019,"48.87834, 2.374525",119,30
2,75120,Paris,Paris 20e Arrondissement,200054781,75,Paris-20e-arrondissement,"55, RUE ORFILA",4.886710e+06,239801.000000,75020,"48.867131, 2.397971",120,55
3,97406,La Plaine-des-Palmistes,La Plaine-des-Palmistes,249740093,974,La Plaine-des-palmistes,None,-2.115291e+01,55.642371,97431,"-21.152906, 55.642371",406,None
4,13005,Aubagne,Aubagne,241300268,13,Aubagne,FENESTRELLES (CHEMIN DE),4.327987e+06,556520.000000,13400,"43.27476, 5.565483",005,None


In [16]:
from sqlalchemy import text
from sqlalchemy.engine import Engine
import pandas as pd
import io

def insert_data_to_db(df: pd.DataFrame, table_name: str, engine: Engine, schema: str):
    """
    Insère un DataFrame dans une table PostgreSQL en utilisant la méthode rapide COPY.
    La fonction vide d'abord la table de destination.

    Args:
        df (pd.DataFrame): Données structurées à insérer.
        table_name (str): Nom de la table cible.
        engine (Engine): Moteur SQLAlchemy connecté à la base PostgreSQL.
        schema (str): Nom du schéma cible.
    """
    if df is None or df.empty:
        print(f"⚠️ Le DataFrame pour '{schema}.{table_name}' est vide. Insertion annulée.")
        return

    print(f"\n🚚 Insertion rapide via COPY dans '{schema}.{table_name}'...")
    
    try:
        # --- 1. Vider la table de destination ---
        with engine.begin() as conn:
            truncate_command = text(f'TRUNCATE TABLE "{schema}"."{table_name}" RESTART IDENTITY CASCADE;')
            conn.execute(truncate_command)
            
        # --- 2. Préparer le buffer CSV en mémoire ---
        buffer_memoire = io.StringIO()
        df.to_csv(buffer_memoire, index=False, header=False, sep=';', na_rep='\\N')
        buffer_memoire.seek(0)
        
        # --- 3. Exécuter la commande COPY ---
        raw_conn = engine.raw_connection()
        cursor = raw_conn.cursor()
        
        sql_copy_command = f"""COPY "{schema}"."{table_name}" FROM STDIN WITH (FORMAT CSV, DELIMITER ';', NULL '\\N')"""
        
        cursor.copy_expert(sql=sql_copy_command, file=buffer_memoire)
        raw_conn.commit()
        
        print(f"-> ✅ {len(df):,} lignes insérées avec succès dans '{schema}.{table_name}'.")

    except Exception as e:
        print(f"❌ Erreur lors de l'insertion dans la table '{schema}.{table_name}': {e}")
        if 'raw_conn' in locals():
            raw_conn.rollback()
        raise
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'raw_conn' in locals() and not raw_conn.closed:
            raw_conn.close()

Dès qu'on a le dataframe prêt à être insérer dans la table on peut utiliser insert_data_to_db :

In [23]:
insert_data_to_db(df_commune, table_name='commune', engine=engine, schema='silver') # Par exemple pour insérer dans la table 'accidents' du schéma 'silver'


🚚 Insertion rapide via COPY dans 'silver.commune'...
❌ Erreur lors de l'insertion dans la table 'silver.commune': null value in column "com_code" of relation "commune" violates not-null constraint
DETAIL:  Failing row contains (null, null, null, null, null, null, RT40, null, null, null, null, 2A085, 40).
CONTEXT:  COPY commune, line 355: "\N;\N;\N;\N;\N;\N;RT40;\N;\N;\N;\N;2A085;40"



NotNullViolation: null value in column "com_code" of relation "commune" violates not-null constraint
DETAIL:  Failing row contains (null, null, null, null, null, null, RT40, null, null, null, null, 2A085, 40).
CONTEXT:  COPY commune, line 355: "\N;\N;\N;\N;\N;\N;RT40;\N;\N;\N;\N;2A085;40"


In [8]:
print(df_commune.columns.tolist())

['com', 'com_code', 'nom_com', 'com_name', 'com_arm_name', 'adr', 'lat', 'long', 'code_postal', 'coordonnees', 'num']


In [7]:
insert_data_to_db(df_vehicule, table_name='vehicule', engine=engine, schema='silver') # Par exemple pour insérer dans la table 'accidents' du schéma 'silver'


🚚 Insertion rapide via COPY dans 'silver.vehicule'...
❌ Erreur lors de l'insertion dans la table 'silver.vehicule': insert or update on table "vehicule" violates foreign key constraint "fk_vehicule_accident"
DETAIL:  Key (num_acc)=(201600055393) is not present in table "accident".



ForeignKeyViolation: insert or update on table "vehicule" violates foreign key constraint "fk_vehicule_accident"
DETAIL:  Key (num_acc)=(201600055393) is not present in table "accident".


In [8]:
# =====================================================================
# FONCTION DE CHARGEMENT UNIVERSELLE (CORRIGÉE)
# =====================================================================
from sqlalchemy.engine import Engine
from sqlalchemy import text  # <--- ASSUREZ-VOUS QUE CET IMPORT EST PRÉSENT
import pandas as pd
import io

def charger_table_silver(df_a_charger: pd.DataFrame, nom_table: str, engine: Engine):
    """
    Charge un DataFrame dans une table spécifiée du schéma 'silver'.
    La fonction vide d'abord la table de destination.
    """
    if df_a_charger is None or df_a_charger.empty:
        print(f"⚠️ Le DataFrame pour la table '{nom_table}' est vide. Le chargement est annulé.")
        return

    print(f"\n🚚 Chargement de 'silver.{nom_table}'...")
    
    try:
        # --- 1. Vider la table de destination ---
        with engine.begin() as conn:
            # LA CORRECTION EST ICI : on enveloppe la commande dans text()
            truncate_command = text(f'TRUNCATE TABLE silver."{nom_table}" RESTART IDENTITY CASCADE;')
            conn.execute(truncate_command)
            
        # --- 2. Charger les données ---
        # On utilise to_sql, qui est simple et suffisant pour les tables de dimensions
        df_a_charger.to_sql(
            nom_table, 
            engine, 
            schema='silver', 
            if_exists='append', 
            index=False, 
            method='multi'
        )
        
        print(f"-> ✅ Table 'silver.{nom_table}' chargée avec succès ({len(df_a_charger):,} lignes).")

    except Exception as e:
        print(f"❌ Erreur lors du chargement de la table '{nom_table}': {e}")
        raise

In [63]:
from sqlalchemy.engine import Engine

# --- Fonction de chargement (corrigée) ---
def charger_table_silver(df_a_charger: pd.DataFrame, nom_table: str, engine: Engine):
    """Charge un DataFrame dans une table spécifiée du schéma 'silver'."""
    print(f"\n🚚 Chargement de 'silver.{nom_table}'...")
    try:
        with engine.begin() as conn:
            # On vide la table avant de la remplir
            truncate_command = text(f'TRUNCATE TABLE silver."{nom_table}" RESTART IDENTITY CASCADE;')
            conn.execute(truncate_command)
        
        # On utilise to_sql pour charger les données
        df_a_charger.to_sql(nom_table, engine, schema='silver', if_exists='append', index=False, method='multi')
        print(f"-> ✅ Table 'silver.{nom_table}' chargée avec {len(df_a_charger):,} lignes.")
    except Exception as e:
        print(f"❌ Erreur lors du chargement de la table '{nom_table}': {e}")
        raise


In [ ]:
charger_table_silver(df_vehicule, 'vehicule', engine)


🚚 Chargement de 'silver.vehicule'...


In [ ]:
charger_table_silver(df_commune, 'commune', engine)